In [1]:
# Advent of Code 2023
# Day 23 Problem 1
import re
from collections import Counter
import functools

with open("aoc_23_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}:\t{line}")

0:	#.#####################
1:	#.......#########...###
2:	#######.#########.#.###
3:	###.....#.>.>.###.#.###
4:	###v#####.#v#.###.#.###
5:	###.>...#.#.#.....#...#
6:	###v###.#.#.#########.#
7:	###...#.#.#.......#...#
8:	#####.#.#.#######.#.###
9:	#.....#.#.#.......#...#
10:	#.#####.#.#.#########v#
11:	#.#...#...#...###...>.#
12:	#.#.#v#######v###.###v#
13:	#...#.>.#...>.>.#.###.#
14:	#####v#.#.###v#.#.###.#
15:	#.....#...#...#.#.#...#
16:	#.#########.###.#.#.###
17:	#...###...#...#...#.###
18:	###.###.#.###v#####v###
19:	#...#...#.#.>.>.#.>.###
20:	#.###.###.#.###.#.#v###
21:	#.....###...###...#...#
22:	#####################.#


In [49]:
from collections import deque

def parse(inp):
    sx, sy = 0, inp[0].index(".")
    tx, ty = len(inp) - 1, inp[-1].index(".")
    return (sx,sy), (tx,ty)

def BFS(G, start, end):
    H, W = len(G), len(G[0])
    q = deque([])
    explored = set([start])

    # [currX, currY, steps, explored]
    q.append([*start, 0, explored])
    scenics = []
    slopes = "^>v<"
    dirs = [(-1,0), (0,1), (1,0), (0,-1)]
    while q: 
        #print(q)
        v = q.popleft()
        if (v[0], v[1]) == end:
            #print(v[2])
            scenics.append(v[3])
            continue

        for i, (dx, dy) in enumerate(dirs):
            xx = v[0] + dx
            yy = v[1] + dy
            if 0 <= xx < H and 0 <= yy < W and (xx,yy) not in v[3] and G[xx][yy] != "#":
                ex = v[3].copy()
                ex.add((xx,yy))
                if G[xx][yy] == slopes[i]:
                    ex.add((xx+dx,yy+dy))
                    q.append([xx+dx, yy+dy, v[2] + 2, ex])
                elif G[xx][yy] == ".":
                    q.append([xx, yy, v[2] + 1, ex])
    #print(scenics)
    winners = sorted([(len(s), s) for s in scenics])
    return winners[-1]

def printPath(G, path):
    print(len(path)-1)
    for i,row in enumerate(G):
        print("".join(["O" if (i,j) in path else row[j] for j,c in enumerate(row)]))

def p1(inp):
    start, end = parse(inp)
    # print(f"Start: {start}\tEnd: {end}")
    best = BFS(inp, start, end)
    # print(best)
    # for p in paths:
    #     printPath(inp, p[1])
    #     print()
    return best[0] - 1


In [50]:
# expected output: 94
p1(TEST)

94

In [51]:
# this takes 20 seconds...
p1(A)

2310

In [52]:
# this isn't halting in 7min so I will retry 

def parseP2(inp):
    for i, row in enumerate(inp):
        tmp = row.replace("^", ".")
        tmp = tmp.replace("v", ".")
        tmp = tmp.replace(">", ".")
        tmp = tmp.replace("<", ".")
        inp[i] = tmp
    sx, sy = 0, inp[0].index(".")
    tx, ty = len(inp) - 1, inp[-1].index(".")
    return inp, (sx,sy), (tx,ty)

def BFSp2(G, start, end):
    H, W = len(G), len(G[0])
    q = deque([])
    explored = set([start])

    # [currX, currY, steps, explored]
    q.append([*start, 0, explored])
    scenics = []
    dirs = [(-1,0), (0,1), (1,0), (0,-1)]
    while q: 
        #print(q)
        v = q.popleft()
        if (v[0], v[1]) == end:
            #print(v[2])
            scenics.append(v[3])
            continue

        for i, (dx, dy) in enumerate(dirs):
            xx = v[0] + dx
            yy = v[1] + dy
            if 0 <= xx < H and 0 <= yy < W and (xx,yy) not in v[3] and G[xx][yy] != "#":
                ex = v[3].copy()
                ex.add((xx,yy))
                q.append([xx, yy, v[2] + 1, ex])
    #print(scenics)
    winners = sorted([(len(s), s) for s in scenics])

def p2(inp):
    G, start, end = parseP2(inp)
    best = BFS(G, start, end)
    return best[0] - 1

In [53]:
p2(TEST)

154

In [177]:
def p2t2(inp):
    SI = SnowIsland(inp)
    SI.printJunctions()
    return SI.DFS()

class SnowIsland:
    def __init__(self, inp) -> None:
        self.baseMap = self.parse(inp)
        self.startNode = Vertex(*self.start)
        self.endNode = Vertex(*self.end)
        self.vertices = set([self.startNode, self.endNode])
        self.edges = set()
        self.transformGraph()

        self.nodeMap = {(v.x, v.y): v for v in self.vertices}
        #print("Pre BFS")
        self.BFS()

        # for v in self.vertices:
        #     v.printNeighbors()
        #     v.printEdges()
    
    def parse(self, inp):
        for i, row in enumerate(inp):
            tmp = row.replace("^", ".")
            tmp = tmp.replace("v", ".")
            tmp = tmp.replace(">", ".")
            tmp = tmp.replace("<", ".")
            inp[i] = tmp
        self.start = (0, inp[0].index("."))
        self.end = (len(inp) - 1, inp[-1].index("."))
        return inp

    def transformGraph(self):
        dirs = [(-1,0), (0,1), (1,0), (0,-1)]
        H, W = len(self.baseMap), len(self.baseMap[0])
        for i, row in enumerate(self.baseMap):
            for j, c in enumerate(row):
                count = 0
                if self.baseMap[i][j] != ".":
                    continue
                for (dx,dy) in dirs:
                    xx = i + dx
                    yy = j + dy
                    if 0 <= xx < H and 0 <= yy < W and self.baseMap[xx][yy] == ".":
                        count += 1
                if count > 2:
                    self.addVertex(i, j)

    def printJunctions(self):
        for i,row in enumerate(self.baseMap):
            print("".join(["x" if (i,j) in self.nodeMap.keys() else " " if row[j] == "#" else row[j] for j,c in enumerate(row)]))

    def addVertex(self, x, y):
        v = Vertex(x, y)
        self.vertices.add(v)

    # for establishing the edges
    def BFS(self):
        H, W = len(self.baseMap), len(self.baseMap[0])
        q = deque([])
        explored = set([self.start])

        # [currX, currY, steps, explored, homeVertex]
        q.append([*self.start, 0, explored, self.startNode])
        while q:
            v = q.popleft()
            coord = (v[0], v[1])
            if coord in self.nodeMap.keys():
                w = self.nodeMap[coord]
                if w not in v[4].neighbors and w != coord and v[2] > 0:
                    #print(f"Adding edge with ({v[4]}, {w}) with weight {v[2]} (remaining elements in queue: {len(q)})")
                    self.addEdge(v[4], w, v[2])
                """ 
                Comment out this break to pass the test input...
                come back and look at these search functions later
                """
                if w == self.endNode:
                    break
                v[4] = self.nodeMap[coord]
                v[2] = 0

            for (dx, dy) in  [(-1,0), (0,1), (1,0), (0,-1)]:
                xx = v[0] + dx
                yy = v[1] + dy
                if 0 <= xx < H and 0 <= yy < W and (xx,yy) not in v[3] and self.baseMap[xx][yy] != "#":
                    ex = v[3].copy()
                    ex.add((xx,yy))
                    q.append([xx, yy, v[2] + 1, ex, v[4]])
    
    # for exploring the traversals
    def DFS(self):
        s = []
        s.append((self.startNode, [self.startNode], 0))
        scores = set()
        while s:
            currNode, visited, score = s.pop()
            if currNode == self.endNode:
                scores.add(score)
                continue
            for e in currNode.edges:
                otherNode = e.v if e.v != currNode else e.w
                if otherNode in visited:
                    continue
                newVisited = visited.copy()
                newVisited.append(otherNode)
                newScore = score + e.weight
                s.append((otherNode, newVisited, newScore))
        return max(list(scores))

    def addEdge(self, v, w, weight):
        e = Edge(v, w, weight)
        v.neighbors.append(w)
        v.edges.append(e)
        w.neighbors.append(v)
        w.edges.append(e)
        self.edges.add(e)

class Vertex:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
        self.neighbors = []
        self.edges = []

    def __str__(self) -> str:
        return f"({self.x},{self.y})"

    def printNeighbors(self):
        print(f"Neighbors of {self}:")
        for n in self.neighbors:
            print(f"\t{n}")

    def printEdges(self):
        print(f"Edges of {self}:")
        for e in self.edges:
            print(f"\t{e.v} --{e.weight}--> {e.w}")

class Edge:
    def __init__(self, v, w, weight) -> None:
        self.v = v
        self.w = w
        self.weight = weight
        self.endpoints = [v, w]

# # #


In [178]:
# exoected output: 154
p2t2(TEST)

 x                     
 .......         ...   
       .         . .   
   ..... ..x..   . .   
   .     . . .   . .   
   x.... . . ..... ... 
   .   . . .         . 
   ... . . ....... ... 
     . . .       . .   
 ..... . . ....... ... 
 .     . . .         . 
 . ... ... ...   ....x 
 . . .       .   .   . 
 ... x.. ....x.. .   . 
     . . .   . . .   . 
 ..... ... ... . . ... 
 .         .   . . .   
 ...   ... ... ... .   
   .   . .   .     .   
 ... ... . ..x.. ..x   
 .   .   . .   . . .   
 .....   ...   ... ... 
                     x 


110

In [179]:
# this finishes in over 2 min if I make a change that doesn't pass the Test input
# so I will want to relook at the searching functions.
p2t2(A)

 x                                                                                                                                           
 ....... ... ...   ... ..... ... ..... ...     ... ............... ... ... ... ....... ....... ...   ...     .......   .........   .......   
       . . . . .   . . .   . . . .   . . .     . . .             . . . . . . . .     . .     . . .   . .     .     .   .       .   .     .   
 ....... . . . . ... . ... . . . ... . . ..... . . ........... ... . . . . . . . ..... ..... ... ..... ... ... ... ..... ....... ... ... ... 
 .       . . . . .   .   . . . .   . . .     . . .           . .   . . . . . . . .         .             . .   . .       .       .   . .   . 
 ... ... . . . . ... ..... ... ... . ... ..... . . ...   ... . ... . ... ... . . ... ..x.. ... ......... . ... . .........   ..... ... ..... 
   . . . . . . .   .             . .     .     . . . .   . . .   . .         . .   . . . .   . .       . .   . .             .     .         
   ...

6738